In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gmr import GMM

from scipy.io.arff import loadarff
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from moebius.model.gmm.evol.optimizer import GMMOptimizer
from moebius.model.gmm.evol.parameters import EvolutionaryConfiguration
from moebius.model.gmm.evol.graph.graph import ProbabilisticGraphShuffler

from moebius.model.gmm.evol.graph.graph import GeneratorNode
from moebius.model.gmm.evol.operators.mutation import mutation, \
    Mutation, \
    ByNameNodeSelector, \
    RandomValueNodeMutator, \
    RandomDeltaNodeMutator, \
    ClapNodeDecoratedMutator, \
    RandomIndexRandomDeltaNodeMutator, \
    RandomValuesSumTo1NodeMutator, \
    DirichletValuesSumTo1NodeMutator

from moebius.model.gmm.evol.operators.crossover import crossover, \
    Crossover, \
    ExchangeFieldNodesCrossover, \
    RandomIndexExchangeFieldNodesCrossover

from moebius.model.gmm.evol.graph.initializer import GMMParametersInitializer, \
    RandomWeightsInitializer, \
    RandomMeansInitializer, \
    RandomCovariancesInitializer, \
    ConstantMeansInitializer, \
    ConstantCovariancesInitializer

from moebius.model.gmm.evol.graph.auto_initializer import AutoMeansInitializer

from moebius.model.gmm.bn import ContinuousBNSerializer, JsonDictSerializer, FilesystemStorage

from moebius.model.gmm.evol.bn import build_bn
from moebius.util import make_diagonal, print_gmm_params_fancy
from moebius import olr, olr_universal

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def explode_cluster(data, target, clusters):
    indices_0_1 = [idx for idx, x in enumerate(target) if x in clusters]
    
    return (
        [list(data[idx]) for idx in indices_0_1],
        [target[idx].decode('UTF-8') if isinstance(target[idx], np.bytes_) else target[idx] for idx in indices_0_1]
    )

In [4]:
data = loadarff('./datasets/openml/volcanoes-a1.arff')

rows = data[0]
meta = data[1]

X = rows[['V1', 'V2', 'V3']]
y = [int(each.decode('UTF-8')) for each in rows['Class']]

In [5]:
X_1_2, y_1_2 = explode_cluster(X, y, (4, 5))

In [6]:
gmm = GMM(n_components=2)
gmm.from_samples(np.array(X_1_2))

w = gmm.priors.tolist()
means = gmm.means.tolist()
cov = make_diagonal(gmm.covariances.tolist())

print_gmm_params_fancy(w, means, cov, decimal_places=3)

 ---  weights   --- 
[0.495 0.505]
 ---   means    --- 
[[3.76626e+02 7.31724e+02 6.13000e-01]
 [6.23400e+02 4.07901e+02 5.83000e-01]]
 --- covariance --- 
[[[ 7.0576529e+04  2.3710779e+04 -6.1580000e+00]
  [ 2.3710779e+04  2.9523721e+04 -6.5980000e+00]
  [-6.1580000e+00 -6.5980000e+00  6.0000000e-03]]

 [[ 5.3298615e+04  1.1976348e+04  1.1108000e+01]
  [ 1.1976348e+04  4.1825613e+04 -2.5530000e+00]
  [ 1.1108000e+01 -2.5530000e+00  1.1000000e-02]]]
 ---    OLR     --- 
0.7978315132386272


In [7]:
pd.DataFrame(data=X_1_2).describe()

,0,1,2
count,174.000000,174.000000,174.000000
mean,501.183908,568.275862,0.597946
std,278.431633,249.607770,0.092360
min,27.000000,51.000000,0.378594
25%,243.750000,371.250000,0.528619
50%,508.000000,565.000000,0.605478
75%,709.750000,799.000000,0.679428
max,1000.000000,1003.000000,0.772831


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_1_2, y_1_2, test_size=0.25, random_state=0)

In [9]:
model = LogisticRegression()
model.fit(X_train, y_train)

accuracy = accuracy_score(y_test, model.predict(X_test))
f1 = f1_score(y_test, model.predict(X_test), pos_label=5)
accuracy, f1

(0.5, 0.47619047619047616)

In [12]:
auto = AutoMeansInitializer.from_data(X_1_2, 2)
auto.create_initial_means('Comp_2', 2)

[[0.6224094924476576], [0.5864018214701798]]

In [13]:
optimizer = GMMOptimizer(
    EvolutionaryConfiguration()
        .set_mutation_types([
            mutation(RandomValueNodeMutator('var', [
                [
                    [400.0, 600.0],
                ],
                [
                    [500.0, 600.0],
                ]
            ])),
            mutation(RandomValuesSumTo1NodeMutator('w')),
        ])
        .set_crossover_types([
            crossover(ExchangeFieldNodesCrossover('var')),
            crossover(RandomIndexExchangeFieldNodesCrossover('var'))
        ]),
    ProbabilisticGraphShuffler(0.5),
    GMMParametersInitializer(
        RandomWeightsInitializer(),
        AutoMeansInitializer.from_data(X_1_2, 2),
        RandomCovariancesInitializer(50, 500)
    )
)

In [25]:
optimization_result = optimizer.optimize(n_dim=3, n_comp=2, target_olr=0.2)

Generations:   0%|          | 1/500 [00:00<?, ?gen/s]

2024-04-14 16:21:10,823 - MultiprocessingDispatcher - Number of used CPU's: 6


 73%|███████▎  | 3654/5000 [00:00<00:00, 18288.31it/s]

2024-04-14 16:21:11,230 - MultiprocessingDispatcher - 1 individuals out of 1 in previous population were evaluated successfully.
2024-04-14 16:21:11,231 - EvoGraphOptimizer - Generation num: 1 size: 1
2024-04-14 16:21:11,231 - EvoGraphOptimizer - Best individuals: HallOfFame archive fitness (1): ['<custom=0.250>']


100%|██████████| 5000/5000 [00:00<00:00, 18304.88it/s]


...

2024-04-14 16:23:17,234 - MultiprocessingDispatcher - 45 individuals out of 54 in previous population were evaluated successfully.
2024-04-14 16:23:17,238 - EvoGraphOptimizer - Generation num: 38 size: 55
2024-04-14 16:23:17,238 - EvoGraphOptimizer - Best individuals: HallOfFame archive fitness (1): ['<custom=0.000>']
2024-04-14 16:23:17,239 - EvoGraphOptimizer - no improvements for 5 iterations
2024-04-14 16:23:17,239 - EvoGraphOptimizer - spent time: 2.1 min


100%|██████████| 5000/5000 [00:00<00:00, 17458.75it/s]


2024-04-14 16:23:17,240 - GroupedCondition - Optimisation finished: Early stopping iterations criteria was satisfied


Generations:   0%|          | 1/500 [02:06<?, ?gen/s]

2024-04-14 16:23:17,241 - OptimisationTimer - Composition time: 2.107 min
2024-04-14 16:23:17,242 - EvoGraphOptimizer - Generation num: 39 size: 1
2024-04-14 16:23:17,242 - EvoGraphOptimizer - Best individuals: HallOfFame archive fitness (1): ['<custom=0.000>']
2024-04-14 16:23:17,243 - EvoGraphOptimizer - no improvements for 6 iterations
2024-04-14 16:23:17,243 - EvoGraphOptimizer - spent time: 2.1 min


In [26]:
optimized_graph = optimization_result[0]
net = build_bn(optimized_graph)

In [27]:
data = net.sample(200)

100%|██████████| 200/200 [00:00<00:00, 1391.05it/s]


In [28]:
data.describe()

,Comp_0,Comp_1,Comp_2
count,200.000000,200.000000,200.000000
mean,529.548844,831.405155,1.106758
std,18.103479,60.371379,23.325214
min,468.948847,427.587552,-59.802589
25%,521.140709,823.877285,-16.006815
50%,531.721460,839.058680,1.506552
75%,541.830914,854.836516,15.714597
max,564.454058,892.758150,66.110737


In [34]:
gmm = GMM(n_components=2)
gmm.from_samples(data.to_numpy())

w = gmm.priors.tolist()
means = gmm.means.tolist()
cov = make_diagonal(gmm.covariances.tolist())

# print_gmm_params_fancy(w, means, cov, decimal_places=3)

olr_universal(w, means, cov)

0.20770187213466382

In [35]:
net

In [37]:
serialized = ContinuousBNSerializer().serialize(net)
j = JsonDictSerializer().serialize(serialized)
FilesystemStorage().write(j, './bns_volcanoes/0.2_0.json')

In [ ]:
gmm = GaussianMixture(n_components = 2) 
gmm.fit(data)
        
sample = gmm.sample(200)
data = sample[0]
labels = [5 if each == 5 else 4 for each in sample[1]]

accuracy = accuracy_score(labels, model.predict(data), pos_label=5)
f1 = f1_score(labels, model.predict(data), pos_label=5)

accuracy, f1